In [17]:
from cimgraph.databases import ConnectionParameters, BlazegraphConnection
from cimgraph.models import NodeBreakerModel, DistributedArea, GraphModel
from cimgraph.models.graph_model import new_mrid #TODO:replace with utils.new_mrid()
import cimgraph.utils as utils
import importlib
from cimgraph import FeederModel

In [18]:
cim_profile = 'cimhub_2023'
cim = importlib.import_module('cimgraph.data_profile.' + cim_profile)

In [19]:
# # RDFLib File Reader Connection
# params = ConnectionParameters(filename="new_model.xml", cim_profile='cimhub_2023', iec61970_301=8)
# rdf = RDFlibConnection(params)

params = ConnectionParameters(url = "http://localhost:8889/bigdata/namespace/kb/sparql", cim_profile=cim_profile, iec61970_301=8)
connection = BlazegraphConnection(params)

In [20]:
def new_substation(connection):

    substation_mrid = new_mrid()
    substation = cim.Substation(mRID = substation_mrid)

    network = DistributedArea(connection=connection, container=substation, distributed=False)

In [10]:
def terminal_to_node(network, terminal:cim.Terminal, node:str|cim.ConnectivityNode):
    if node.__class__ == str:
        for node_obj in network.graph[cim.ConnectivityNode].values():
            if node_obj.name == node:
                terminal.ConnectivityNode = node_obj
                node_obj.Terminals.append(terminal)
    else:
        terminal.ConnectivityNode = node
        node.Terminals.append(terminal)


In [15]:
def new_disconnector(network:DistributedArea, container:cim.EquipmentContainer, name:str, 
                node1:str|cim.ConnectivityNode, node2:str|cim.ConnectivityNode,
                open:bool=False, normalOpen:bool=False, retained:bool=True) -> None:

    disconnector = cim.Disconnector(name = name, mRID = new_mrid())
    t1 = cim.Terminal(name=f"{name}.1", mRID = new_mrid(), sequenceNumber=1)
    t2 = cim.Terminal(name=f"{name}.2", mRID = new_mrid(), sequenceNumber=2)

    terminal_to_node(network, t1, node1)
    terminal_to_node(network, t2, node2)

    disconnector.EquipmentContainer = container
    disconnector.open = open
    disconnector.normalOpen = normalOpen
    disconnector.Terminals.append(t1)
    disconnector.Terminals.append(t2)
    
    t1.ConductingEquipment = disconnector
    t2.ConductingEquipment = disconnector

    network.add_to_graph(disconnector)
    network.add_to_graph(t1)
    network.add_to_graph(t2)


In [16]:
def new_breaker(network:DistributedArea, container:cim.EquipmentContainer, name:str, 
                node1:str|cim.ConnectivityNode, node2:str|cim.ConnectivityNode,
                open:bool=False, normalOpen:bool=False, retained:bool=True) -> None:

    breaker = cim.Breaker(name = name, mRID = new_mrid())
    t1 = cim.Terminal(name=f"{name}.1", mRID = new_mrid(), sequenceNumber=1)
    t2 = cim.Terminal(name=f"{name}.2", mRID = new_mrid(), sequenceNumber=2)

    terminal_to_node(network, t1, node1)
    terminal_to_node(network, t2, node2)

    breaker.EquipmentContainer = container
    breaker.open = open
    breaker.normalOpen = normalOpen
    breaker.Terminals.append(t1)
    breaker.Terminals.append(t2)
    
    t1.ConductingEquipment = breaker
    t2.ConductingEquipment = breaker

    network.add_to_graph(breaker)
    network.add_to_graph(t1)
    network.add_to_graph(t2)

    #TODO: new_discrete()



In [26]:
def new_bus_tie(network:GraphModel, substation:cim.Substation, main_bus:cim.ConnectivityNode, transfer_bus:cim.ConnectivityNode):

    junction1 = cim.ConnectivityNode(name=f"{substation.name}_bt_j1", mRID = new_mrid(), ConnectivityNodeContainer=substation)
    junction2 = cim.ConnectivityNode(name=f"{substation.name}_bt_j2", mRID = new_mrid(), ConnectivityNodeContainer=substation)
    airgap1 = new_disconnector(network, substation, name = f'{substation.name}_bt1', node1 = main_bus, node2 = junction1)
    bus_tie = new_breaker(network, substation, name = f"{substation.name}_bus_tie", node1 = junction1, node2 = junction2)
    airgap2 = new_disconnector(network, substation, name = f'{substation.name}_bt1', node1 = junction2, node2 = transfer_bus)
    network.add_to_graph(junction1)
    network.add_to_graph(junction2)
    
    

In [39]:
from cimgraph import GraphModel
import cimgraph.data_profile.cimhub_2023 as cim

def new_branch(network:GraphModel, substation:cim.Substation, main_bus:cim.ConnectivityNode, transfer_bus:cim.ConnectivityNode, 
               branch:cim.ConductingEquipment, sequenceNumber:int):

    junction1 = cim.ConnectivityNode(name=f"{substation.name}_{sequenceNumber}_j1", mRID = new_mrid(), ConnectivityNodeContainer=substation)
    junction2 = cim.ConnectivityNode(name=f"{substation.name}_{sequenceNumber}_j2", mRID = new_mrid(), ConnectivityNodeContainer=substation)
    junction3 = cim.ConnectivityNode(name=f"{substation.name}_{sequenceNumber}_j3", mRID = new_mrid(), ConnectivityNodeContainer=substation)

    breaker = new_breaker(network, substation, name = f"{substation.name}_{sequenceNumber}", node1 = junction1, node2 = junction2)

    airgap1 = new_disconnector(network, substation, name = f'{substation.name}_{sequenceNumber+1}', node1 = main_bus, node2 = junction1)
    airgap2 = new_disconnector(network, substation, name = f'{substation.name}_{sequenceNumber+2}', node1 = junction2, node2 = junction3)
    airgap3 = new_disconnector(network, substation, name = f'{substation.name}_{sequenceNumber+3}', node1 = junction3, node2 = transfer_bus)

    for terminal in branch.Terminals:
        if int(terminal.sequenceNumber) == 1:
            terminal.ConnectivityNode = junction3
        
    
    network.add_to_graph(junction1)
    network.add_to_graph(junction2)
    network.add_to_graph(junction3)

In [34]:
def new_main_and_transfer(network:GraphModel = None, name:str = "new_sub"):
    
    # create substation
    substation_mrid = new_mrid()
    substation = cim.Substation(mRID = substation_mrid, name=name)

    if not network:
        network = DistributedArea(connection=connection, container=substation, distributed=False)

       

    # main bus
    main_bus = cim.ConnectivityNode(name="main_bus", mRID=new_mrid())
    main_bus.ConnectivityNodeContainer = substation
    network.add_to_graph(main_bus)

    # transfer bus
    transfer_bus = cim.ConnectivityNode(name="transfer_bus", mRID=new_mrid())
    transfer_bus.ConnectivityNodeContainer = substation
    network.add_to_graph(transfer_bus)

    # create bus_tie
    new_bus_tie(network, substation, main_bus, transfer_bus)

    return network

In [35]:
network = new_main_and_transfer()

In [37]:
network.pprint(cim.ConnectivityNode)

{
    "8569b7ee-9cde-47cf-b520-883f99f5616f": {
        "mRID": "8569b7ee-9cde-47cf-b520-883f99f5616f",
        "name": "main_bus",
        "ConnectivityNodeContainer": "5e7027ae-fb70-46cb-8683-3a5131392214",
        "Terminals": "['c0a562fe-5ed7-41e5-bdc3-1b838aece1a7']"
    },
    "bb25f54e-4745-4b96-9c83-6b377ed9a9b5": {
        "mRID": "bb25f54e-4745-4b96-9c83-6b377ed9a9b5",
        "name": "transfer_bus",
        "ConnectivityNodeContainer": "5e7027ae-fb70-46cb-8683-3a5131392214",
        "Terminals": "['4a027ce0-f77c-4a5c-92f8-4e2d0593fa0c']"
    },
    "cd379ee6-385c-49f8-8794-60c80f694249": {
        "mRID": "cd379ee6-385c-49f8-8794-60c80f694249",
        "name": "new_sub_bt_j1",
        "ConnectivityNodeContainer": "5e7027ae-fb70-46cb-8683-3a5131392214",
        "Terminals": "['8f45aaa2-0581-4d09-aa45-0af6dd4c96c9', '6734cb45-8b89-4b38-869a-a3d441cad35e']"
    },
    "e63aa60a-bb1d-4c57-9415-b706f29934b7": {
        "mRID": "e63aa60a-bb1d-4c57-9415-b706f29934b7",
        "name